Programa com codificaçao RLE

In [ ]:
import numpy as np
from PIL import Image

# Função para compressão RLE
def compressao_rle(imagem_ppm):
    altura, largura, _ = imagem_ppm.shape
    resultado = [largura, altura]

    for canal in range(3):
        for linha in range(altura):
            pixels = imagem_ppm[linha, :, canal]
            i = 0
            while i < len(pixels):
                inicio = i
                while i + 1 < len(pixels) and pixels[i] == pixels[i + 1]:
                    i += 1
                comprimento = i - inicio + 1

                if comprimento > 1:
                    while comprimento > 127:
                        resultado.append(127)
                        resultado.append(pixels[inicio])
                        comprimento -= 127
                    resultado.append(comprimento)
                    resultado.append(pixels[inicio])
                else:
                    inicio_diferentes = i
                    while i + 1 < len(pixels) and pixels[i] != pixels[i + 1]:
                        i += 1
                    i += 1
                    comprimento = i - inicio_diferentes
                    resultado.append(-comprimento)
                    resultado.extend(pixels[inicio_diferentes:i])
            i += 1

    return resultado

# Função para descompressão RLE
def descompressao_rle(dados):
    largura = dados.pop(0)
    altura = dados.pop(0)
    imagem = np.zeros((altura, largura, 3), dtype=np.uint8)

    for canal in range(3):
        for linha in range(altura):
            linha_pixels = []
            while len(linha_pixels) < largura:
                contador = dados.pop(0)
                if contador > 0:
                    valor = dados.pop(0)
                    linha_pixels.extend([valor] * contador)
                elif contador < 0:
                    comprimento = -contador
                    linha_pixels.extend(dados[:comprimento])
                    dados = dados[comprimento:]
            imagem[linha, :, canal] = linha_pixels[:largura]

    return imagem

# Teste com imagem PPM
imagem = Image.open("EntradaRGB.ppm")
imagem_array = np.array(imagem)

dados_comprimidos = compressao_rle(imagem_array)
with open("imagem_comprimida.rle", "w") as f:
    f.write(" ".join(map(str, dados_comprimidos)))

with open("imagem_comprimida.rle", "r") as f:
    dados_comprimidos = list(map(int, f.read().split()))
imagem_descomprimida = descompressao_rle(dados_comprimidos)

imagem_recuperada = Image.fromarray(imagem_descomprimida)
imagem_recuperada.save("imagem_descomprimida.ppm")
